# Trabalho 2 de Análise Multivariada
Versão: 2024/10/09
Autores: Gabriela Dellamora, Leonardo Ripes e Luize Iensse

In [77]:
import pandas as pd
import numpy as np
import math

# Questão 1 
a) (0.5) Considerando a v.a. (X, Y) com distribuição de [Poisson bivariada](https://filipezabala.com/ea/distr-contagem.html#poisson-bivariada), obtenha λˆ1, λˆ2 e λˆ3 por máxima verossimilhança.

b) (0.5) A partir do modelo obtido no item a, calcule P(X=2, Y=3). Compare com a probabilidade obtida através da proporção amostral da tabela acima.

c) (0.5) A partir do modelo obtido no item a, calcule E(X), E(Y), V(X) e V(Y).

d) (0.5) Calcule P(X≤2, Y≥3). Compare com a proporção amostral.

In [38]:
data = np.array([
    [24, 28, 12, 6, 0, 1, 0, 0],
    [19, 39, 41, 23, 1, 0, 0, 0],
    [12, 45, 47, 29, 12, 3, 1, 0],
    [7, 17, 32, 21, 14, 1, 0, 0],
    [2, 2, 3, 11, 13, 1, 2, 1],
    [0, 0, 3, 6, 4, 2, 2, 1],
    [0, 0, 2, 0, 2, 4, 0, 1],
    [0, 0, 0, 1, 2, 0, 0, 0],
])
# Frequências marginais
marginal_y = np.sum(data, axis=1)
marginal_x = np.sum(data, axis=0)
total = np.sum(marginal_x)
print(f"Marginal X = {marginal_x}")
print(f"Marginal Y = {marginal_y}")
print(f"Total = {total}")


Marginal X = [ 64 131 140  97  48  12   5   3]
Marginal Y = [ 71 123 149  92  35  18   9   3]
Total = 500


In [60]:
# Questão A

# λ1, λ2
lambda_1 = np.sum(marginal_x) / len(marginal_x)  # λ1
lambda_2 = np.sum(marginal_y) / len(marginal_y)  # λ2

print(f"λ1={lambda_1:.2f}")
print(f"λ2={lambda_2:.2f}")

# λ3
# médias ponderadas
mean_x = np.mean(np.repeat(np.arange(len(marginal_x)), marginal_x))
mean_y = np.mean(np.repeat(np.arange(len(marginal_y)), marginal_y))

# covariância = 1 / N (E(X, Y) - (E(X) * E(Y))
cov_xy = np.sum((np.repeat(np.arange(len(marginal_x)), marginal_x) - mean_x) *
                 (np.repeat(np.arange(len(marginal_y)), marginal_y) - mean_y)) / total
lambda_3 = cov_xy

'''em R:
#> cov_xy <- cov(X, Y)
#> cov_xy
# [1] 1.92982
'''

print(f"λ3={lambda_3:.2f}")

λ1=62.50
λ2=62.50
λ3=1.93


In [61]:
# Questão B

# P(X = x, Y = y)
'''calcula a probabilidade bivariada de Poisson'''
def poisson_bivariate_prob(x, y, lambda_1, lambda_2, lambda_3):
    sum_term = 0
    for k in range(0, min(x, y) + 1):
        sum_term += (math.comb(x, k) * math.comb(y, k) * math.factorial(k) *
                     (lambda_3 / (lambda_1 * lambda_2))**k)
    prob = (lambda_1**x * lambda_2**y * math.exp(-lambda_1 - lambda_2 - lambda_3) /
            (math.factorial(x) * math.factorial(y))) * sum_term
    return prob

# P(X = 2, Y = 3)
prob_model = poisson_bivariate_prob(2, 3, lambda_1, lambda_2, lambda_3)

# proporção amostral
n_23 = data[2, 3]  # Contagem de X = 2 e Y = 3
total_obs = np.sum(data)

prob_sample = n_23 / total_obs

# Comparação dos resultados
difference = abs(prob_model - prob_sample)

''' EM R:
> prob_model <- dbvpois(2, 3, a, b, c)
> print(paste("Probabilidade pela Poisson Bivariada: ", prob_model))
[1] "Probabilidade pela Poisson Bivariada:  5.97839559506662e-48"
> # proporção amostral
> n_23 <- data[3, 4]
> N <- sum(data)
> prob_sample <- n_23 / N
> print(paste("Proporção amostral P(X=2, Y=3): ", prob_sample))
[1] "Proporção amostral P(X=2, Y=3):  0.058"
> # Comparar os resultados
> difference <- abs(prob_model - prob_sample)
> print(paste("Diferença entre o modelo e a proporção amostral: ", difference))
[1] "Diferença entre o modelo e a proporção amostral:  0.058"
'''

print(f"Probabilidade pelo modelo de Poisson Bivariada={prob_model}")
print(f"Proporção amostral={prob_sample}")
print(f"Diferença entre modelo e proporção amostral={difference}")



Probabilidade pelo modelo de Poisson Bivariada=6.001479123174308e-48
Proporção amostral=0.058
Diferença entre modelo e proporção amostral=0.058


In [63]:
# Questão C

E_X = lambda_1 + lambda_3
E_Y = lambda_2 + lambda_3

V_X = lambda_1 + lambda_3
V_Y = lambda_2 + lambda_3

E_X, E_Y, V_X, V_Y
print(f"E(X)={E_X} : E(Y)={E_Y} : V(X)={V_X} : V(Y)={V_Y}")

E(X)=64.42596 : E(Y)=64.42596 : V(X)=64.42596 : V(Y)=64.42596


In [66]:
# Calcular P(X ≤ 2, Y ≥ 3)
total_prob = 0
for x in range(3):       # X <= 2
    for y in range(3, 8):  # Y >= 3
        total_prob += poisson_bivariate_prob(x, y, lambda_1, lambda_2, lambda_3)

# Calcular a proporção amostral
data_points = [(x, y) for x in range(3) for y in range(3, 8)]
total_count = sum(data[x, y] for x, y in data_points)
sample_proportion = total_count / total_obs
'''em R
> total_prob <- 0
> for (x in 0:2) {  # X <= 2
+   for (y in 3:7) {  # Y >= 3
+     total_prob <- total_prob + dbvpois(x, y, a, b, c)
[1] "Probabilidade pelo modelo de Poisson Bivariada P(X <= 2, Y >= 3):  1.26465241621426e-43"


> total_count <- sum(data[1:3, 4:8])  # Somando para X <= 2 e Y >= 3
> N <- sum(data)  # Total de observações
> prob_sample <- total_count / N
> print(paste("Proporção amostral P(X <= 2, Y >= 3): ", prob_sample))
[1] "Proporção amostral P(X <= 2, Y >= 3):  0.152"

> difference <- abs(total_prob - prob_sample)
> print(paste("Diferença entre o modelo e a proporção amostral: ", difference))
[1] "Diferença entre o modelo e a proporção amostral:  0.152"
'''

print(f"Probabilidade pelo Modelo de Poisson Bivariada={total_prob}")
print(f"Proporção Amostral={sample_proportion}")
print(f"Diferença={abs(total_prob - sample_proportion)}")

Probabilidade pelo Modelo de Poisson Bivariada=1.2695260335622792e-43
Proporção Amostral=0.152
Diferença 0.152


# Questão 2
Com base nas informações do estudo e considerando que 7 pessoas forem escolhidas aleatoriamente, responda:

a) (1.0) Qual a probabilidade de que uma tenha sangue do tipo A, uma do B, duas do AB e três do O?

b) (1.0) Qual a probabilidade de que todas tenham sangue do tipo A?

c) (1.0) Qual a probabilidade de que todas tenham o mesmo tipo de sangue?

In [69]:
# Questão A

n = 7
p_A = 0.35
p_B = 0.12
p_AB = 0.04
p_O = 0.49

k_A = 1
k_B = 1
k_AB = 2
k_O = 3

# probabilidade multinomial
prob = (math.factorial(n) / (math.factorial(k_A) * math.factorial(k_B) * math.factorial(k_AB) * math.factorial(k_O))) * \
       (p_A**k_A) * (p_B**k_B) * (p_AB**k_AB) * (p_O**k_O)

print(f"A probabilidade de que exatamente uma pessoa tenha sangue do tipo A, uma do tipo B, duas do tipo AB e três do tipo O é {prob}")

A probabilidade de que exatamente uma pessoa tenha sangue do tipo A, uma do tipo B, duas do tipo AB e três do tipo O é 0.003320525376


In [71]:
# Questão B
k_A = 7
k_B = 0
k_AB = 0
k_O = 0

prob = (math.factorial(n) / (math.factorial(k_A) * math.factorial(k_B) * math.factorial(k_AB) * math.factorial(k_O))) * \
       (p_A**k_A) * (p_B**k_B) * (p_AB**k_AB) * (p_O**k_O)

print(f"A probabilidade de que todas as 7 pessoas tenham sangue do tipo A é aproximadamente {prob}")

A probabilidade de que todas as 7 pessoas tenham sangue do tipo A é aproximadamente 0.0006433929687499997


In [73]:
prob_all_same = (p_A ** n) + (p_B ** n) + (p_AB ** n) + (p_O ** n)
prob_all_same
print(f"A probabilidade de que todas as 7 pessoas tenham o mesmo tipo de sangue é {prob_all_same}")

A probabilidade de que todas as 7 pessoas tenham o mesmo tipo de sangue é 0.007425982179159999
